In [1]:
import numpy as np
import arcpy
from arcpy import sa
from arcpy.sa import *

In [ ]:
## Maps for quantiles 5% and 95% 
arcpy.CheckOutExtension("Spatial")

npp_layer = arcpy.Raster("_/npp__steppe_reclass_eco.tif") # NPP data for each year
mask_raster = arcpy.Raster("_/ecoregions.tif") # mask of ecoregions
npp_layer = sa.ExtractByMask(npp_layer, mask_raster)
npp_array = arcpy.RasterToNumPyArray(npp_layer)

# Exclude NoData
no_data_value = npp_layer.noDataValue
print(f"NoData: {no_data_value}")

if no_data_value is not None:
    mask = (npp_array != 0) & (npp_array != no_data_value)
else:
    mask = (npp_array != 0)
   
npp_array_masked = np.where(mask, npp_array, np.nan)
print(f"Number of elements after masking: {np.count_nonzero(~np.isnan(npp_array_masked))}")

if np.isnan(np.nanmin(npp_array_masked)):
    raise ValueError("Data array includes only NaN values")

min_value = np.nanmin(npp_array_masked)
max_value = np.nanmax(npp_array_masked)
print(f"Min value: {min_value}, Max value: {max_value}")
q5 = np.nanpercentile(npp_array_masked, 5)
q95 = np.nanpercentile(npp_array_masked, 95)
print(f"5 quantile: {q5}, 95 quantile: {q95}")

if q5 >= q95:
    raise ValueError(f"5 qiantile is equal to or greater than 95 quantile: {q5} >= {q95}")

remap = RemapRange([
    [float(min_value), float(q5 + 1e-6), 1], # 5% to 1 class
    [float(q5 + 1e-6), float(q95), 2],       # 2 class - 5-95%
    [float(q95), float(max_value), 3]        # 3 class - 95%
])

quantile_npp2001 = Reclassify(npp_layer, "VALUE", remap, "NODATA")
final_raster_path = "_/quantile_npp_.tif"
quantile_npp2001.save(final_raster_path)

arcpy.CheckInExtension("Spatial")
arcpy.Delete_management(npp_layer)